In [2]:
import numpy as np
import tensorflow as tf
import random
import dqn

/Users/bychkim/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ModuleNotFoundError: No module named 'dqn'

In [3]:
from collections import deque
import gym
env = gym.make('CartPole-v0')

In [4]:
# Constants defining our neural network
# Input and output siza based on the Env
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

dis = 0.9
REPLAY_MEMORY = 50000

In [5]:
class DQN:
    def __init__(self, session, input_size, output_size, name="main"):
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        
        self._build_network()
    
    def _build_network(self, h_size=10, l_rate=1e-1):
        with tf.variable_scope(self.net_name):
            self._X = tf.placeholder(
                tf.float32, [None, self.input_size], name="input_x") # None에 입력을 해서 n개까지 입력 받을 수 있도록
            
            # First layer of weights
            W1 = tf.get_variable("W1", shape=[self.input_size, h_size],
                                 initializer=tf.contrib.layers.xavier_initializer())
            layer1 = tf.nn.tanh(tf.matmul(self._X, W1))
            
            # Second layer of weights
            W2 = tf.get_variable("W2", shape=[h_size, self.output_size],
                                 initializer=tf.contrib.layers.xavier_initializer())
            
            # Q prediction
            self._Qperd = tf.matmul(layer1, W2)
            
        # We need to define the parts of the network needed for learning a policy
        self._Y = tf.placeholder(
            shape=[None, self.output_size], dtype=tf.float32)
        
        # Loss function
        self._loss = tf.reduce_mean(tf.square(self._Y - self._Qpred))
        
        # Learning
        self._train = tf.train.AdamOptimizer(
            learning_rate=l_rate).minimize(self._loss)
        
    def predict(self, state):
        x = np.reshape(state, [1, self.input_size])
        return self.sesstion.run(self._Qpred, feed_dict={self._X: x})
    
    # network를 직접 학습시키지 않고, 따로 변수를 지정하여 학습을 시키는데 도움을 주는 함수
    def update(self, x_stack, y_stack):
        return self.session.run([self._loss, self._train], feed_dict={self._X: x_stack, self._Y: y_stack})

In [6]:
def replay_train(mainDQN, targetDQN, train_batch):
    x_stack = np.empty(0).reshape(0, input_size)
    y_stack = np.empty(0).reshape(0, output_size)
    
    # Get stored information from the buffer
    for state, action, reward, next_state, done in train_batch:
        Q = mainDQN.predict(state)
        
        # terminal?
        if done:
            Q[0, action] = reward
        else:
            # Get target from target DQN (Q`)
            # Obtain the Q` values by feeding the new state through our network
            Q[0, action] = reward + dis * np.max(DQN.predict(next_state))
            
        y_stack = np.vstack([y_stack, Q]) # vertical stack
        x_stack = np.vstack([x_stack, state])
        
    # Train our network using target and predicted Q values on each episode
    return mainDQN.update(x_stack, y_stack)

In [7]:
def get_copy_var_ops(*, dest_scope_name="target", src_scope_name="main"):
    # Copy variables src_scope to dest_scope
    op_holder = []
    
    src_vars = tf.get_collection(
        tf.GraphKeys.TRANINABLE_VARIABLES, scope=src_scope_name)
    dest_vars = tf.get_collection(
        tf.GraphKeys.TRANINABLE_VARIABLES, scope=dest_scope_name)
    
    for src_var, dest_var in zip(src_vars, dest_vars):
        op_holder.append(dest_var.assign(src_var.value())) # tensorflow에 있는 assign을 사용 ; (tensor).assign
        
    return op_holder

In [8]:
def bot_play(mainDQN):
    # See our trained network in action
    s = env.reset()
    reward_sum = 0
    while True:
        env.render()
        a = np.argmax(mainDQN.predict(s))
        s, reward, done, _ = env.step(a)
        reward_sum += reward
        if done:
            print("Total score: {}".format(reward_sum))
            break

In [9]:
def main():
    max_episodes = 5000
    
    # Store the previous observations in replay memory
    replay_buffer = deque()
    
    with tf.Session() as sess:
        mainDQN = dqn.DQN(sess, input_size, output_size, name="main")
        targetDQN = dqn.DQN(sess, input_size, output_size, name="target")
        tf.global_variables_initializer().run()
        
        # Initial copy q_net -> target_net
        copy_ops = get_copy_var_ops(dest_scope_name="target", src_scope_name="main")
        
        sess.run(copy_ops) # targetDQN == mainDQN
        
        # mainDQN만 update / targetDQN은 update되지 않는다 (copy)
        
        
        for episode in range(max_episodes):
            e = 1.0 / ((episode / 10) + 1)
            done = False
            step_count = 0
            
            state = env.reset()
            
            while not done:
                if np.random.rand(1) < e:
                    action = env.action_space.sample()
                else:
                    # Choose an action by greedily from the Q-network
                    action = np.argmax(mainDQN.predict(state))
                    
                # Get new state and reward from environment
                next_state, reward, done, _ = env.step(action)
                if done: # big penalty
                    reward = -100
                
                # Save the experience to our buffer
                replay_buffer.append((state, action, reward, next_state, done))
                if len(replay_buffer) > REPLAY_MEMORY:
                    replay_buffer.popleft()
                
                state = next_state
                step_count += 1
                if step_count > 10000: # Good enough
                    break
                    
            print("Episode: {} steps: {}".format(episode, step_count))
            if step_count > 10000:
                pass
                # break
            if episode % 10 == 1: # train every 10 episodes
                # Get a random batch of experiences.
                for _ in range(50):
                    # Minibatch works better
                    minibatch = random.sample(replay_buffer, 10)
                    loss, _ = simple_replay_train(mainDQN, minibatch)
                print("Loss: ", loss)
                
                # Copy q_net -> target_net
                sess.run(copy_ops)
        
        # See our trained bot in action
        env2 = wrapper.Monitor(env, 'gym-results', force=True)
        
        for i in range(200):
            bot_play(mainDQN, env=env2)
            
        env2.close()
        gym.upload("gym-results", api_key="sk_VT2wPcSSOylnlPORltmQ")

if __name__ == "__main__":
    main()

NameError: name 'dqn' is not defined

# Recap

1. network 2개 생성 (main & target)

2. target = main_net (copy; 학습이 시작되기 전에 같게 만들어 줌)

3. env 환경을 주고(생성), 

    - loop 실행 : action -> state, reward ...
    
    - buff D에 저장 
    
    - sampling and training 
    
    - Y를 build할 때 target_network를 사용 
    
    - Y를 이용해서 main_network를 update
    
    - 10번에 한번씩 main과 target이 같도록 복사


# Exercise I

* Hyper parameter tuning
    - Learning rate
    - Sample size
    - Decay factor
    
* Network structure (Fully Connected network을 사용했지만, RNN 이나 CNN도 사용 가능)
    - add bias
    - (activation function) test tanh, sigmoid, relu, etc.
    - improve TF network to reduce sess.run() calls

* Reward redesign
    - 1, 1, 1, 1, 1, ... , -100
    - 1, 0.9, 0.99, ... , 0 

# Exercise II

* Simple block based car race?
    - https://github.com/golbin/TensorFlow-Tutorials/tree/master/07%20-%20DQN
    - DQN 2013? -> 2015 version으로 강화
    
* Rewrite it using DQN 2015 algorithm?

# Exercise III

* DQN implementations
    - https://github.com/songrotek/DQN-Atari-Tensorflow
    - https://github.com/dennybritz/reinforcement-learning/blob/master/DQN/dqn.py
    - https://github.com/devsisters/DQN-tensorflow
    
    
* Other games
    - https://www.ehrenbrav.com/2016/08/teaching-your-computer-to-play-super-mario-bros-a-fork-of-the-google-deepmind-atari-machine-learning-project/
    
    
* RMA approach
    - Run
    - Modify
    - Adapt (to your new game/problems)